In [2]:
# The optimization of share price prediction model based on support vector machine, 2011

In [51]:
import yfinance as yf
import numpy as np
import pandas as pd

from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import train_test as tt

In [74]:
def MAPE(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100 

In [75]:
####################### 논문 재현 ###################

In [81]:
price_data = yf.download("BESTEAST.BO",start = '2010-09-01', end = "2010-12-18")
price_data = price_data[["Open","High","Low","Close","Volume"]]

lag_n = 3
price_data[["Open","High","Low","Volume"]] = price_data[["Open","High","Low","Volume"]].shift(lag_n)
price_data = price_data[lag_n:len(price_data)]

[*********************100%***********************]  1 of 1 completed


/tmp/ipykernel_1083611/677405865.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_data[["Open","High","Low","Volume"]] = price_data[["Open","High","Low","Volume"]].shift(lag_n)


In [82]:
x = price_data[["Open","High","Low","Volume"]]
y = price_data[["Close"]]

scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

x = pd.DataFrame(scaler_x.fit_transform(x))
y = pd.DataFrame(scaler_y.fit_transform(y))

train_n = 60
train_x = x[0:train_n]
train_y = y[0:train_n]

test_x = x[train_n:len(price_data)]
test_y = y[train_n:len(price_data)]


In [83]:
model = SVR(kernel = 'rbf', C=102.2, gamma = 0.0625, epsilon= 0.01)
model.fit(train_x, train_y)

/home/whfhrs3260/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=102.2, epsilon=0.01, gamma=0.0625)

In [84]:
svr_pred = np.reshape(model.predict(test_x),(len(model.predict(test_x)),1))

svr_pred = scaler_y.inverse_transform(svr_pred)
test_y = scaler_y.inverse_transform(test_y)

svr_mae = mean_absolute_error(test_y, svr_pred)
svr_mse = mean_squared_error(test_y, svr_pred)
svr_rmse = np.sqrt(mean_squared_error(test_y, svr_pred))
svr_mape = MAPE(test_y, svr_pred)

In [85]:
svr_rmse, svr_mse, svr_mae, svr_mape

(1.6813534843125009, 2.826949539209787, 1.4086947075168845, 11.798792057730575)

In [43]:
######################### 논문 재현 다우존스 ######################

In [45]:
price_data = yf.download("^DJI",start = '2010-09-01', end = "2010-12-18")
price_data = price_data[["Open","High","Low","Close","Volume"]]

lag_n = 3
price_data[["Open","High","Low","Volume"]] = price_data[["Open","High","Low","Volume"]].shift(lag_n)
price_data = price_data[lag_n:len(price_data)]

train_n = 60
train_x = price_data[["Open","High","Low","Volume"]][0:train_n]
train_y = price_data[["Close"]][0:train_n]

test_x = price_data[["Open","High","Low","Volume"]][train_n:len(price_data)]
test_y = price_data[["Close"]][train_n:len(price_data)]

model = SVR(kernel = 'rbf', C=102.2, gamma = 0.0625, epsilon= 0.01)
model.fit(train_x, train_y)

svr_pred = np.reshape(model.predict(test_x),(len(model.predict(test_x)),1))

svr_mae = mean_absolute_error(test_y, svr_pred)
svr_mse = mean_squared_error(test_y, svr_pred)
svr_rmse = np.sqrt(mean_squared_error(test_y, svr_pred))
svr_mape = MAPE(test_y, svr_pred)

[*********************100%***********************]  1 of 1 completed


/tmp/ipykernel_1083611/696687641.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_data[["Open","High","Low","Volume"]] = price_data[["Open","High","Low","Volume"]].shift(lag_n)
/home/whfhrs3260/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/whfhrs3260/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3430: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=a

In [46]:
svr_rmse, svr_mse, svr_mae, svr_mape

(368.5732480687675,
 135846.2391919612,
 362.72438333151246,
 Close    3.177996
 dtype: float64)

In [13]:
########################## (news, past_close) 2 variable 결과 ##################

In [34]:
# price data, window = 1
dir = "/home/whfhrs3260/csv_data/price_data_score_10years.csv"
stock = "^DJI"
variable = ["before_close","Score"]
window_size = 1
start_date = "2012-01-01"
end_date = "2022-04-30"

scaler_x, scaler_y, x, y, x_t, y_t = tt.train_test_result(dir, stock, variable, window_size, start_date, end_date)

[*********************100%***********************]  1 of 1 completed


/home/whfhrs3260/Python/10year_data/review/train_test.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  set_news_data_date.price_date[i]=standard  + datetime.timedelta(days=1)


[*********************100%***********************]  1 of 1 completed


/home/whfhrs3260/Python/10year_data/review/train_test.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_data.before_close[i+1] = price_data.Close[i]


In [35]:

x = x.reshape(len(x),window_size*len(variable))
y = y.reshape(len(y),1)
x_t = x_t.reshape(len(x_t),window_size*len(variable))
y_t = y_t.reshape(len(y_t),1)

model = SVR(kernel = 'rbf', C=102.2, gamma = 0.0625, epsilon= 0.01)
model.fit(x, y)


#예측값
svr_pred = np.reshape(model.predict(x_t),(len(model.predict(x_t)),1))

svr_pred = scaler_y.inverse_transform(svr_pred)
y_t = scaler_y.inverse_transform(y_t)
    
svr_mae = mean_absolute_error(y_t, svr_pred)
svr_mse = mean_squared_error(y_t, svr_pred)
svr_rmse = np.sqrt(mean_squared_error(y_t, svr_pred))
svr_mape = MAPE(y_t, svr_pred)

/home/whfhrs3260/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [36]:
svr_rmse, svr_mse, svr_mae, svr_mape

(419.72512148306237,
 176169.17760397148,
 296.44836919586527,
 1.0205721915419257)